# P3D Web notebook
<u>Version</u> : 0.2 (02/06/2021)

In [50]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code of P3D Web is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [51]:
# -----------------------------------------------------------------------------------------------------------------
# Package
# -----------------------------------------------------------------------------------------------------------------

from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 55em; }</style>"))
%matplotlib notebook
import numpy as np
import sympy as sp
import cxroots as cx
import scipy.special as spsp
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.cm as cm
import ipywidgets as widgets
from IPython.display import display
from IPython.display import Code
from IPython.display import Markdown


# -----------------------------------------------------------------------------------------------------------------
# Computation function for back-end
# -----------------------------------------------------------------------------------------------------------------

def compute_root_spectrum(n,m,s0,value_tau):
    
    IntProgress_mid_generic.value = 3
    s = sp.symbols('s')  # define variable s for our problem to be solved
    tau = sp.symbols('tau')  # define variable tau : delay
    a = sp.symbols(["a{:d}".format(i) for i in range(n)], real=True)
    alpha = sp.symbols(["alpha{:d}".format(i) for i in range(m + 1)], real=True)
    Polynomial = s**n + np.array(a).dot([s**i for i in range(n)]) # Revient à faire s^n + a_{n-1}^{n-1}...
    Delayed = np.array(alpha).dot([s**i for i in range(m+1)])*sp.exp(-s*tau) # Revient à faire 
    Q = Polynomial + Delayed 
    SysDerivatif = [Q]
    IntProgress_mid_generic.value = 4
    for i in range(n+m+1):
        DerniereDerivee = SysDerivatif[-1]
        SysDerivatif.append(DerniereDerivee.diff(s)) # Dérivée par rapport à s
    IntProgress_mid_generic.value = 5
    sol = sp.linsolve(SysDerivatif[:-1], alpha + a).args[0] # Solveur selon les alpha et les a
    solNum = sol.subs({s : s0})
    solNum = solNum.subs({tau : value_tau})
    a_num = list(solNum[m + 1:])
    alpha_num = list(solNum[:m + 1])
    QNumerique = s**n + np.array(a_num).dot([s**i for i in range(n)])+\
    np.array(alpha_num).dot([s**i for i in range(m+1)])*sp.exp(-s*tau)
    IntProgress_mid_generic.value = 6
    QNumerique = QNumerique.subs(tau, value_tau)
    sysRootFinding = [QNumerique, QNumerique.diff(s)]
    sysFunc = [sp.lambdify(s, i) for i in sysRootFinding]
    IntProgress_mid_generic.value = 7
    rect = cx.Rectangle([-100, 10], [-100, 100])
    roots = rect.roots(sysFunc[0], sysFunc[1], rootErrTol=1e-5, absTol=1e-5, M = n + m + 1)
    xroot = np.real(roots[0])
    yroot = np.imag(roots[0])
    IntProgress_mid_generic.value = 8
    return xroot,yroot,QNumerique

# -----------------------------------------------------------------------------------------------------------------

def compute_admissibilite(n,m,value_s0,value_tau,a0,a1):
    
    s = sp.symbols('s')  # define variable s for our problem to be solved
    tau = sp.symbols('tau')  # define variable tau : delay

    a = sp.symbols(["a{:d}".format(i) for i in range(n)], real=True)
    alpha = sp.symbols(["alpha{:d}".format(i) for i in range(m + 1)], real=True)
    avalue = [a0, a1]

    Polynomial = s**n + np.array(a).dot([s**i for i in range(n)]) # Revient à faire s^n + a_{n-1}^{n-1}...
    Delayed = np.array(alpha).dot([s**i for i in range(m+1)])*sp.exp(-s*tau) # Revient à faire 
    #b^m*s^m + b_{m-1}^{m-1}...
    Q = Polynomial + Delayed 

    SysDerivatif = [Q]
    for i in range(m+1):
        DerniereDerivee = SysDerivatif[-1]
        SysDerivatif.append(DerniereDerivee.diff(s)) # Dérivée par rapport à s

    sol = sp.linsolve(SysDerivatif[:-1], alpha).args[0] # Solveur selon les alpha et les a

    finaleq = SysDerivatif[-1].subs({alph : alphacoef for alph, alphacoef in zip(alpha, sol)}) #remplace les coeffs
    finaleq = finaleq.subs({asymb: aval for asymb, aval in zip(a, avalue)})
    solS0 = finaleq.subs({tau : value_tau})
    solS0 = sp.solve(solS0)
    solS0eval = [i.evalf() for i in solS0]

    computedS0 = solS0[1]
    alpha_num = sol.subs({asymb: aval for asymb, aval in zip(a, avalue)})
    alpha_num = alpha_num.subs({s : computedS0})
    alpha_num = alpha_num.subs({tau : value_tau})
    alpha_num_eval = [i.evalf() for i in alpha_num]
    alpha_sens = alpha_num_eval

    finaleq = SysDerivatif[-1].subs({alph : alphacoef for alph, alphacoef in zip(alpha, sol)}) #remplace les coeffs
    finaleq = finaleq.subs({asymb: aval for asymb, aval in zip(a, avalue)})
    solTau = finaleq.subs({s : value_s0})
    solTau = sp.solve(solTau)

    computedTau = solTau[0]
    alpha_num = sol.subs({asymb: aval for asymb, aval in zip(a, avalue)})
    alpha_num = alpha_num.subs({tau : computedTau})
    alpha_num = alpha_num.subs({s : value_s0})
    alpha_num_eval = [i.evalf() for i in alpha_num]

    polyAdm = SysDerivatif[-1].subs({alph : alphacoef for alph, alphacoef in zip(alpha, sol)})
    polyAdm = polyAdm.subs({asymb: aval for asymb, aval in zip(a, avalue)})
    polyAdm = sp.simplify(polyAdm)

    s0range = np.arange(-10, 0, 0.01)
    taurange = np.arange(0, 10, 0.01)

    func = sp.lambdify([s, tau], polyAdm)

    return s0range,taurange,polyAdm,s,tau,Q,avalue,alpha_sens,SysDerivatif,alpha,a

def getRoots(m, Q, dominancy, delay, avalue, alphavalue, xwindow, ywindow,s,a,alpha,tau):
    derivees = [Q, Q.diff(s)]
    for i in range(len(derivees)) :
        derivees[i] = derivees[i].subs({ai: ai_num for ai, ai_num in zip(a, avalue)})
        derivees[i] = derivees[i].subs({alphai: alphai_num for alphai, alphai_num in zip(alpha, alphavalue)})
        derivees[i] = derivees[i].subs({tau : delay})
    func = [sp.lambdify(s, i) for i in derivees]
    rect = cx.Rectangle(xwindow, ywindow)
    root_count = rect.count_roots(func[0])
    roots = rect.roots(func[0], func[1], rootErrTol=1e-5, absTol=1e-5, M=m+2)
    xroot, yroot = np.real(roots[0]), np.imag(roots[0])
    return xroot, yroot

def solve_tau_connu(tau_val, acoef, Q,m,s,SysDerivatif,alpha,a,tau):
    sys = [Q]
    for i in range(m+1):
        DerniereDerivee = sys[-1]
        sys.append(DerniereDerivee.diff(s)) # Dérivée par rapport à s
    sol = sp.linsolve(SysDerivatif[:-1], alpha).args[0]
    finaleq = SysDerivatif[-1].subs({alph : alphacoef for alph, alphacoef in zip(alpha, sol)}) 
    finaleq = finaleq.subs({asymb: aval for asymb, aval in zip(a, acoef)})
    solS0 = finaleq.subs({tau : tau_val})
    solS0 = sp.solve(solS0)
    solS0eval = [i.evalf() for i in solS0]
    try :
        solution = max([i for i in solS0eval if i<0])
    except Exception :
        traceback.print_exc()
    return solution

def compute_sensibilite(value_tau,Q,m,s,SysDerivatif,alpha,a,tau):
    tau_nominal = value_tau
    step = 1e-2
    nbIt = 10
    values = [0] + [-step * i for i in range(1, nbIt + 1)] + [step * i for i in range(1, nbIt + 1)]
    values.sort()
    tau_sens = []
    s0_sens = []
    for value in values :
        tau_sens.append(tau_nominal + value)
        s0_sens.append(solve_tau_connu(tau_sens[-1], [2, 1], Q,m,s,SysDerivatif,alpha,a,tau))
        
    sensIterations = len(tau_sens)
    bList = np.linspace(start=255, stop=0, num=sensIterations//2) #Blue to black
    rList = np.linspace(start=0, stop=255, num=sensIterations//2) #black to red
    if len(bList)+len(rList)==sensIterations-1 :
        zerosToFill = [0] * (sensIterations//2 + 1)
    else :
        zerosToFill = [0] * (sensIterations//2 )
        
    #bList = [*bList, *zerosToFill]
    #rList = [*zerosToFill, *rList]
    #colorArray = []
    #for i in range(len(tau_sens)):
    #    colorArray.append([rList[i], 0, bList[i]])
    #colorArray = np.array(colorArray)
    
    normaliser = clr.Normalize(min(tau_sens), max(tau_sens))
    colormapper = cm.ScalarMappable(norm = normaliser , cmap = plt.get_cmap ("coolwarm") )
    colormapper.set_array(tau_sens)

    return tau_sens,s0_sens,colormapper

def factorization_integral_latex(n, m, s0, tau):
    factor = str(tau**(m+1) / spsp.factorial(m))
    parenthesis = "(s + " + str(-s0) + ")"
    power = "^{" + str(n + m + 1) + "}"
    
    return r"\$" + "\Delta(s) = " + factor + parenthesis + power + r"\int_0^1 t^{" + str(m) + r"} (1 - t)^{" + str(n) + "} e^{-" + str(tau) + "t" + parenthesis + "} \mathrm{d}t" + "$"

def factorization_1f1_latex(n, m, s0, tau):
    factor = str(tau**(m+1) * spsp.factorial(n) / spsp.factorial(n+m+1))
    parenthesis = "(s + " + str(-s0) + ")"
    power = "^{" + str(n + m + 1) + "}"
    
    return r"\$" + "\Delta(s) = " + factor + parenthesis + power + r" {}_1 F_1(" + str(m+1) + r", " + str(n+m+2) + ", -" + str(tau) + parenthesis + ")" + "$" 

# -----------------------------------------------------------------------------------------------------------------
# Output
# -----------------------------------------------------------------------------------------------------------------

output_mid_generic = widgets.Output()
output_mid_co = widgets.Output()
output_crrid_generic = widgets.Output()

# -----------------------------------------------------------------------------------------------------------------

output_root_spectrum_computation_mid_generic = widgets.Output()
output_output_equation_computation_mid_generic = widgets.Output()
output_factorized_integral_equation_computation_mid_generic = widgets.Output()
output_factorization_1f1_equation_computation_mid_generic = widgets.Output()

# -----------------------------------------------------------------------------------------------------------------

output_admissibilite_computation_mid_co = widgets.Output()
output_output_equation_computation_mid_co = widgets.Output()
output_sensibilite_computation_mid_co = widgets.Output()

# -----------------------------------------------------------------------------------------------------------------
# User Interface input
# -----------------------------------------------------------------------------------------------------------------

slider_n_mid_generic = widgets.IntSlider(min=0,max=3,step=1,description='n :',value=2)
slider_m_mid_generic = widgets.IntSlider(min=0,max=3,step=1,description='m :',value=1)

FloatText_s0_mid_generic = widgets.BoundedFloatText(min=-10.0,max=0.0,step=0.1,description='s0 :',value=-3.0,disabled=False)
FloatText_tau_mid_generic = widgets.BoundedFloatText(min=0.50,max=2.00,step=0.01,description='tau :',value=0.73,disabled=False)

layout_button_ready_to_compute_mid_generic = widgets.Layout( width='100%')
layout_IntProgress_mid_generic = widgets.Layout( width='75%')
layout_valid_statut_mid_generic = widgets.Layout( width='99%') #, border='2px solid #8e2042'

button_ready_to_compute_mid_generic = widgets.Button(description='Compute',layout = layout_button_ready_to_compute_mid_generic, disabled=False,button_style='success',icon='check-circle')
IntProgress_mid_generic = widgets.IntProgress(layout = layout_IntProgress_mid_generic,  value=0,min=0,max=10,description='Loading:',bar_style='success',orientation='horizontal') # style={'bar_color': '#8e2042'}, 'success', 'info', 'warning', 'danger' or ''
valid_statut_mid_generic = widgets.Valid(value=False,description='Status :',layout = layout_valid_statut_mid_generic)

layout_validate_mid_generic = widgets.AppLayout(header=None,left_sidebar=button_ready_to_compute_mid_generic,center=IntProgress_mid_generic,right_sidebar=valid_statut_mid_generic,footer=None)

# -----------------------------------------------------------------------------------------------------------------

slider_n_mid_co = widgets.IntSlider(min=0,max=3,step=1,description='n :',value=2)
slider_m_mid_co = widgets.IntSlider(min=0,max=3,step=1,description='m :',value=1)

ToggleButtons_mid_co = widgets.ToggleButtons(options=['s0', 'tau'], description ='parameter',disabled=False,button_style='',orientation='horizontal',tooltips=['s0 is known', 'tau is known'])
ToggleButtons_mid_co.style.button_width='100%'


FloatText_s0_mid_co = widgets.BoundedFloatText(min=-10.0,max=0.0,step=0.1,description='value :',value=-3.0,disabled=False)
FloatText_tau_mid_co = widgets.BoundedFloatText(min=0.50,max=2.00,step=0.01,description='tau :',value=0.73,disabled=False)

IntText_a0_mid_co = widgets.BoundedIntText(min=-10,max=10,step=1,description='a0 :',value=1,disabled=False)
IntText_a1_mid_co = widgets.BoundedIntText(min=-10,max=10,step=1,description='a1 :',value=1,disabled=False)

layout_button_ready_to_compute_mid_co = widgets.Layout( width='100%')
layout_IntProgress_mid_co = widgets.Layout( width='75%')
layout_valid_statut_mid_co = widgets.Layout( width='99%') #, border='2px solid #8e2042'

button_ready_to_compute_mid_co = widgets.Button(description='Compute',layout = layout_button_ready_to_compute_mid_co, disabled=False,button_style='success',icon='check-circle')
IntProgress_mid_co = widgets.IntProgress(layout = layout_IntProgress_mid_co,  value=0,min=0,max=10,description='Loading:',bar_style='success',orientation='horizontal') # style={'bar_color': '#8e2042'}, 'success', 'info', 'warning', 'danger' or ''
valid_statut_mid_co = widgets.Valid(value=False,description='Status :',layout = layout_valid_statut_mid_co)

layout_validate_mid_co = widgets.AppLayout(header=None,left_sidebar=button_ready_to_compute_mid_co, center=IntProgress_mid_co, right_sidebar=valid_statut_mid_co,footer=None)

# -----------------------------------------------------------------------------------------------------------------
# Computation function for front-end
# -----------------------------------------------------------------------------------------------------------------

def computation_mid_generic(n,m,s0,value_tau):
    
    output_root_spectrum_computation_mid_generic.clear_output()
    output_output_equation_computation_mid_generic.clear_output()
    output_factorized_integral_equation_computation_mid_generic.clear_output()
    output_factorization_1f1_equation_computation_mid_generic.clear_output()
    
    with output_root_spectrum_computation_mid_generic:
        
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.set_title("Root spectrum")
        ax.set_xlabel(r"Re$(s)$")
        ax.set_ylabel(r"Im$(s)$")
        ax.axhline(linewidth=2, color='black', zorder = 2)
        ax.axvline(linewidth=2, color='black', zorder = 2)
        ax.axvspan(0, 1, alpha=0.5, color='red', zorder = 2)
        ax.grid()
        line, = ax.plot([], [], 'o', color='steelblue', zorder = 3)
        
        xr,yr,eq = compute_root_spectrum(n,m,s0,value_tau)
        line.set_data(xr, yr)
        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw()
        IntProgress_mid_generic.value = 9
    
    with output_output_equation_computation_mid_generic:
        print ("Fp = ",eq)
    
    with output_factorized_integral_equation_computation_mid_generic:
        factorization_integral_latex_equation_mid_generic = factorization_integral_latex(n, m, s0, value_tau)
        display(Markdown(factorization_integral_latex_equation_mid_generic))
    
    with output_factorization_1f1_equation_computation_mid_generic:
        factorization_1f1_latex_equation_mid_generic = factorization_1f1_latex(n, m, s0, value_tau)
        display(Markdown(factorization_1f1_latex_equation_mid_generic))
        IntProgress_mid_generic.value = 10

def button_ready_to_compute_clicked_mid_generic(self):
    
    valid_statut_mid_generic.value = False
    IntProgress_mid_generic.value = 1
    
    n_mid_generic = slider_n_mid_generic.value
    m_mid_generic = slider_m_mid_generic.value
    s0_mid_generic = FloatText_s0_mid_generic.value
    tau_mid_generic = FloatText_tau_mid_generic.value
    
    IntProgress_mid_generic.value = 2
    
    computation_mid_generic(n_mid_generic,m_mid_generic,s0_mid_generic,tau_mid_generic)
    
    IntProgress_mid_generic.value = 10
    valid_statut_mid_generic.value = True
    
button_ready_to_compute_mid_generic.on_click(button_ready_to_compute_clicked_mid_generic)

# -----------------------------------------------------------------------------------------------------------------


def computation_mid_co(n,m,value_s0,value_tau,a0,a1):
        
    output_admissibilite_computation_mid_co.clear_output()
    output_output_equation_computation_mid_co.clear_output()
    output_sensibilite_computation_mid_co.clear_output()
    
    with output_admissibilite_computation_mid_co:
        
        # additional calculations for the plot 
        IntProgress_mid_co.value = 4
        s0range,taurange,polyAdm,s,tau,Q,avalue,alpha_sens,SysDerivatif,alpha,a = compute_admissibilite(n,m,value_s0,value_tau,a0,a1)
        IntProgress_mid_co.value = 5
        s0range = np.arange(-10, 0, 0.01)
        taurange = np.arange(0, 10, 0.01)

        func = sp.lambdify([s, tau], polyAdm)
    
        fig, ax = plt.subplots()
        X, Y = np.meshgrid(s0range, taurange)
        z = func(X, Y)
        CS = ax.contour(X, Y, z, [0])
        ax.grid()
        plt.xlabel(r"$s_0$")
        plt.ylabel(r"$\tau$")
        plt.title("Admissibility plot")
    
        
    with output_output_equation_computation_mid_co :
        IntProgress_mid_co.value = 6
        print ("Eq = ",polyAdm)
    
    with output_sensibilite_computation_mid_co :
                
        # additional calculations for the plot
        IntProgress_mid_co.value = 7
        tau_sens,s0_sens,colormapper = compute_sensibilite(value_tau,Q,m,s,SysDerivatif,alpha,a,tau)
        IntProgress_mid_co.value = 8
        
        scaler = lambda x : [i / 255 for i in x]
        fig,ax = plt.subplots()

        for i in range(len(tau_sens)):
            xroot, yroot = getRoots(1, Q, s0_sens[i], tau_sens[i],avalue, alpha_sens, [-5, 5], [-5, 5],s,a,alpha,tau)
            ax.scatter(xroot, yroot, c=len(xroot)*[colormapper.to_rgba(tau_sens[i])])
            
        IntProgress_mid_co.value = 9
        ax.grid()
        ax.set_axisbelow(True)
        fig.colorbar(colormapper)

        ax.set_xlabel(r"$Re(s)$")
        ax.set_ylabel(r"$Im(s)$")
        ax.set_title(r"Sensitivity plot, $\tau \in $"+f"[{tau_sens[0]}, {tau_sens[-1]}]")
        tau_sens.sort()
        
        IntProgress_mid_co.value = 10
        valid_statut_mid_co.value = True
    
def button_ready_to_compute_clicked_mid_co(self):
    
    valid_statut_mid_co.value = False
    IntProgress_mid_co.value = 1
    
    n_mid_co = slider_n_mid_co.value
    m_mid_co = slider_m_mid_co.value
    s0_mid_co = FloatText_s0_mid_co.value
    tau_mid_co = FloatText_tau_mid_co.value
    
    a0_mid_co = IntText_a0_mid_co.value
    a1_mid_co = IntText_a1_mid_co.value
    
    IntProgress_mid_co.value = 2
    
    computation_mid_co(n_mid_co,m_mid_co,s0_mid_co,tau_mid_co,a0_mid_co,a1_mid_co)
    
    
    
button_ready_to_compute_mid_co.on_click(button_ready_to_compute_clicked_mid_co)

# -----------------------------------------------------------------------------------------------------------------
# User Interface output
# -----------------------------------------------------------------------------------------------------------------

box_input_mid_generic = widgets.HBox([slider_n_mid_generic, slider_m_mid_generic, FloatText_s0_mid_generic, FloatText_tau_mid_generic])
box_input_mid_co = widgets.HBox([slider_n_mid_co, slider_m_mid_co, ToggleButtons_mid_co, FloatText_s0_mid_co])
box_input_2_mid_co = widgets.HBox([IntText_a0_mid_co,IntText_a1_mid_co])

accordion_mid_generic = widgets.Accordion(children=[output_root_spectrum_computation_mid_generic, output_output_equation_computation_mid_generic, output_factorized_integral_equation_computation_mid_generic, output_factorization_1f1_equation_computation_mid_generic])
accordion_mid_generic.set_title(0, 'Root spectrum')
accordion_mid_generic.set_title(1, 'Output equation')
accordion_mid_generic.set_title(2, 'Factorized integral equation')
accordion_mid_generic.set_title(3, 'Hypergeometric factorization')

accordion_mid_co = widgets.Accordion(children=[output_admissibilite_computation_mid_co, output_output_equation_computation_mid_co, output_sensibilite_computation_mid_co])
accordion_mid_co.set_title(0, 'Admissibility plot')
accordion_mid_co.set_title(1, 'Output equation')
accordion_mid_co.set_title(2, 'Sensitivity plot')

description_label_mid_generic = widgets.Label('Insert degree of polynomial n, degree of delay polynomial m, rightmost root s0 and delay tau, then push the button to compute : ')
box_mid_generic = widgets.VBox([description_label_mid_generic, box_input_mid_generic,output_mid_generic, layout_validate_mid_generic, accordion_mid_generic ])

description_label_mid_co = widgets.Label('Insert degree of polynomial n, degree of delay polynomial m, and select between rightmost root s0 or delay tau : ')
description_label_2_mid_co = widgets.Label('Insert system coefficients a0 and a1, then push the button to compute : ')
box_mid_co = widgets.VBox([description_label_mid_co, box_input_mid_co, description_label_2_mid_co, box_input_2_mid_co, output_mid_co, layout_validate_mid_co, accordion_mid_co ])

description_label_crrid_generic = widgets.Label('Insert degree of polynomial n, degree of delay polynomial m, rightmost root s0 and delay tau, then push the button to compute : ')
box_crrid_generic = widgets.VBox([description_label_crrid_generic,output_crrid_generic])

tab = widgets.Tab([box_mid_generic, box_mid_co, box_crrid_generic])
tab.set_title(0, 'MID Generic')
tab.set_title(1, 'MID Control oriented')
tab.set_title(2, 'CRRID Generic')

file = open("images/header_p3d_web.png", "rb")
image = file.read()
header_image = widgets.Image(value=image,format='png',width=980,height=25)

dashboard = widgets.VBox([header_image,tab])
display(dashboard)